In [1]:
# Задание 2
# Обучите T-learner c помощью двух логистических регрессий (LogisticRegression). 
# Для этого вам потребуются данные А/Б теста.


import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklift.metrics import uplift_auc_score, qini_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv("ab_results.csv")

# разделим данные на признаки и целевую переменную
X = data.drop(['target'], axis=1)  # все столбцы, кроме целевой переменной
y = data['target']  # целевая переменная (например, количество поездок)

# разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                    stratify=data[['target', 'treatment']],
                                    random_state=42)

# разделяем обучающую выборку на treatment=1 и treatment=0
X_train_treat = X_train[X_train['treatment'] == 1].drop('treatment', axis=1)
y_train_treat = y_train[X_train['treatment'] == 1]

X_train_control = X_train[X_train['treatment'] == 0].drop('treatment', axis=1)
y_train_control = y_train[X_train['treatment'] == 0]

# создаём две логистические регрессии
t_learner_1 = LogisticRegression(random_state=1, max_iter=1000)
t_learner_0 = LogisticRegression(random_state=1, max_iter=1000)

# обучаем модели
t_learner_1.fit(X_train_treat, y_train_treat)
t_learner_0.fit(X_train_control, y_train_control)


# применяем обе модели к тестовой выборке (без признака 'treatment')
X_test_features = X_test.drop('treatment', axis=1)
df_test = X_test.copy()
df_test['yes_treatment'] = t_learner_1.predict_proba(X_test_features)[:, 1]
df_test['no_treatment'] = t_learner_0.predict_proba(X_test_features)[:, 1]

# расчёт uplift
uplift_vals = df_test['yes_treatment'] - df_test['no_treatment']
df_test['uplift_score'] = uplift_vals

# метрики uplift score и qini score
uplift_score = uplift_auc_score(y_test.values, uplift_vals.values, X_test['treatment'].values)
qini_score = qini_auc_score(y_test.values, uplift_vals.values, X_test['treatment'].values)

print(f"Uplift AUC: {uplift_score:.2f}")
print(f"Qini AUC: {qini_score:.2f}")

/home/mle-user/mle_projects/mle-sprint5-uplift/.uplift_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Uplift AUC: 0.13
Qini AUC: 0.16


/home/mle-user/mle_projects/mle-sprint5-uplift/.uplift_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [2]:
# Здесь всё очень похоже на обучение S-learner: вы также используете базовую модель (например, RandomForestClassifier или логистическую регрессию), 
# но обучаете две независимые — одну только на данных с treatment=1 (группа воздействия), а вторую — только на данных с treatment=0 (контрольная группа). 
# В библиотеке causalml есть класс BaseTClassifier, который реализует T-learner для различных моделей. 
# После обучения обеих моделей вы получаете uplift для каждого пользователя как разницу между предсказаниями этих моделей.
# 
# Задание 4
# Посмотрим на код обучения T-learner с помощью случайного леса из библиотеки causalml. 

from causalml.inference.meta import BaseTClassifier
from sklearn.ensemble import RandomForestClassifier
from sklift.metrics import uplift_auc_score, qini_auc_score

# создаём базовые модели для тестовой и контрольной групп 
treatment_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=6,
    random_state=42
)

control_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=6,
    random_state=42
)

# инициализируем T-learner
t_learner = BaseTClassifier(
    treatment_learner=treatment_model,
    control_learner=control_model,
    control_name=0,  # название контрольной группы в столбце treatment
)

# обучаем T-learner
# примечание: X_train должен иметь столбец 'treatment' со значениями 'treatment' и 'control'
t_learner.fit(
    X=X_train.values,  # Признаки без столбца treatment
    treatment=X_train['treatment'].values,       # Столбец с признаком воздействия
    y=y_train.values                            # Целевая переменная
)

# получаем предсказания для тестовой выборки
uplift_pred = t_learner.predict(X_test.values)

# рассчитываем метрики uplift
uplift_score = uplift_auc_score(
    y_test.values, 
    uplift_pred.squeeze(), 
    X_test['treatment'].values
)
qini_score = qini_auc_score(
    y_test.values, 
    uplift_pred.squeeze(), 
    X_test['treatment'].values
)

print(f"Uplift AUC: {uplift_score:.2f}")
print(f"Qini AUC: {qini_score:.2f}")

Uplift AUC: 0.16
Qini AUC: 0.20


In [3]:
# Задание 6
# Ваша задача — реализовать T-learner и получить показатели не хуже, чем у S-learner: uplift score не ниже 0.20 и Qini score не ниже 0.15. 
# Датасет уже был разделен на обучающую и тестовую выборки: 
#   X_train, y_train — обучающая выборка;
#   X_test, y_test — тестовая выборка;
# В X_train и X_test есть столбец treatment, который указывает, получил ли пользователь скидку (1) или нет (0).
# Вы можете использовать любой из изученных подходов (например, RandomForestClassifier, LogisticRegression, UpliftRandomForestClassifier, CatBoostClassifier, BaseTClassifier). 
# Главное — достичь требуемого качества. Пробуйте различные модели и комбинации гиперпараметров.
# Реализуйте обучение двух отдельных моделей: одну для treatment-группы (получивших скидку), другую — для control-группы (без скидки). 
# Затем рассчитайте uplift для тестовой выборки и выведите значения метрик uplift AUC и Qini AUC. 
# Чтобы было проще, используйте библиотеку causalml. 
# Но если хочется реализовать алгоритм «вручную» — мы не против! Данные можно скачать здесь.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from causalml.inference.meta import BaseTClassifier
from sklift.metrics import uplift_auc_score, qini_auc_score

# загрузка данных
data = pd.read_csv("yandex_plus.csv")

# разделим данные на признаки и целевую переменную
X = data.drop(['conversion'], axis=1)  
y = data['conversion']  

# применяем маппинг для удобства обучения модели
treatment_mapping = {
    'control': 0,  # 0 соответствует контрольной группе
    'treatment1': 1  # 1 соответствует группе воздействия
}

X['treatment'] = X['treatment'].map(treatment_mapping)

# разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                    stratify=data[['conversion', 'treatment']],
                                    random_state=42)

# создаём базовые модели для тестовой и контрольной групп 
treatment_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    random_state=42
)

control_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    random_state=42
)

# инициализируем T-learner
t_learner = BaseTClassifier(
    treatment_learner=treatment_model,
    control_learner=control_model,
    control_name=0,  # название контрольной группы в столбце treatment
)

# обучаем T-learner
# примечание: X_train должен иметь столбец treatment со значениями treatment и control
t_learner.fit(
    X=X_train.values,  
    treatment=X_train['treatment'].values,       # Столбец с признаком воздействия
    y=y_train.values                            # Целевая переменная
)

# получаем предсказания для тестовой выборки
uplift_pred = t_learner.predict(X_test.values)

# рассчитываем метрики uplift
uplift_score = uplift_auc_score(
    y_test.values, 
    uplift_pred.squeeze(), 
    X_test['treatment'].values
)
qini_score = qini_auc_score(
    y_test.values, 
    uplift_pred.squeeze(), 
    X_test['treatment'].values
)

print(f"Uplift AUC: {uplift_score:.2f}")
print(f"Qini AUC: {qini_score:.2f}")

Uplift AUC: 0.24
Qini AUC: 0.17
